In [2]:
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import random
import torchvision
from pathlib import Path

In [3]:
class Ageset(Dataset):

    def __init__(self, path, transforms = None, valid=False, split_pct = 0.3):
        self.image_paths = list(Path(path).rglob("*.png"))
        random.seed(42)

        random.shuffle(self.image_paths)
        split_point = int(len(self)*0.3)
        if valid:
            self.image_paths = self.image_paths[:split_point]
        else:
            self.image_paths = self.image_paths[split_point:]
    def __len__(self):
        return len(self.image_paths)

    def imgpath_to_tensor(self, imgpath):
        return transforms.PILToTensor()(Image.open(imgpath)).float()

    def __getitem__(self,i):
        # return self.image_paths[i]
        if isinstance(i, slice):
            return [self[n] for n,_ in enumerate(self.image_paths[i])]
        return (self.imgpath_to_tensor(self.image_paths[i]),
                int(self.image_paths[i].parent.name))

In [4]:
class adaptedRes(nn.Module):
    def __init__(self):
        super(adaptedRes, self).__init__()
        resnet = torchvision.models.resnet50(pretrained=True)
        modules=list(resnet.children())[:-1]
        self.resnet =nn.Sequential(*modules)
        self.fc = nn.Linear(in_features=2048, out_features=1, bias=True)

        # for m in self.modules():
        # if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(self.fc.weight)

    def forward(self,x):
        out = self.resnet(x)
        x = torch.flatten(out, 1)
        return self.fc(x)

In [72]:
NUM_EPOCH = 10
LR = 0.0005
BATCH_SIZE = 32
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_set = Ageset("data/face_age")[:200]
train_dl = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

test_set = Ageset("data/face_age", valid=True)[:200]
test_dl = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

model = adaptedRes()
model.to(DEVICE)
opt = torch.optim.Adam(model.parameters(), LR)

def mae_loss(y, pred):
    return (torch.abs(y-pred.T)).mean()
loss_fn = mae_loss

for epoch in range(NUM_EPOCH):
    print('epoch start')
    model.train()
    total_loss_train = 0
    for data in train_dl:
        x, y = data[0].to(DEVICE), data[1].to(DEVICE)
        opt.zero_grad()
        pred = model(x)
        loss = loss_fn(y, pred)
        total_loss_train += loss * len(y)
        loss.backward()
        opt.step()
    model.eval()
    total_loss_test = 0
    with torch.no_grad():
        for data in test_dl:
            x, y = data[0].to(DEVICE), data[1].to(DEVICE)
            total_loss_test += loss_fn(y, model(x)) * len(y)
    print(f'loss after epoch {epoch}: {total_loss_train/len(train_set)}, {total_loss_test/len(test_set)}')
    


epoch start
loss after epoch 0: 27.021869659423828, 21.833492279052734
epoch start


KeyboardInterrupt: 

In [71]:
running = 0
for i in test_set[:10]:
    preds = i[1],model(i[0][None].to(DEVICE)).item()
    print(f'target {preds[0]}, predicted {preds[1]}')
    loss = abs(preds[0]-preds[1])
    # print('loss',loss)
    running += loss
print(running, running/2)

target 60, predicted 62.526214599609375
target 18, predicted 17.39681625366211
target 38, predicted 26.99531364440918
target 7, predicted 8.993149757385254
target 13, predicted 11.819043159484863
target 12, predicted 11.703676223754883
target 65, predicted 76.5073013305664
target 4, predicted 4.940330505371094
target 15, predicted 17.17656135559082
target 5, predicted 6.174081802368164
33.40279006958008 16.70139503479004
